In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Folder with outcome files
folder_path = 'robustness_results'

# Loop through files
for filename in os.listdir(folder_path):
    if '_outcomes_' in filename and filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)

        # Ensure required columns exist
        if all(col in df.columns for col in ['Expected Annual Damage', 'Expected Number of Deaths', 'Evacuation Costs']):
            # Plot setup
            fig = plt.figure(figsize=(10, 8))
            gs = gridspec.GridSpec(2, 2, width_ratios=[4, 1], height_ratios=[1, 4], hspace=0.05, wspace=0.05)

            # Scatter plot
            ax_scatter = fig.add_subplot(gs[1, 0])
            ax_scatter.scatter(
                df['Expected Annual Damage'],
                df['Expected Number of Deaths'],
                color='blue',
                alpha=0.05,  # Low opacity for dense plots
                s=10
            )
            ax_scatter.set_xlabel('Expected Annual Damage')
            ax_scatter.set_ylabel('Expected Number of Deaths')
            ax_scatter.grid(True)

            # Top histogram
            ax_hist_x = fig.add_subplot(gs[0, 0], sharex=ax_scatter)
            ax_hist_x.hist(df['Expected Annual Damage'], bins=30, color='gray', alpha=0.7)
            ax_hist_x.axis('off')

            # Right histogram
            ax_hist_y = fig.add_subplot(gs[1, 1], sharey=ax_scatter)
            ax_hist_y.hist(df['Expected Number of Deaths'], bins=30, orientation='horizontal', color='gray', alpha=0.7)
            ax_hist_y.axis('off')

            # Title
            fig.suptitle(f'{filename}', fontsize=10)

            # Layout and show
            plt.show()

        else:
            print(f"Skipping {filename}: missing required columns.")


In [ ]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import matplotlib.cm as cm
import numpy as np


folder_path = 'robustness_results'


# Generate a sequential colormap from warm (orange/red) to cool (blue)
policy_colors = ['#b40426ff', '#f7b89cff', '#90ee90ff', '#3b4cc0ff']

color_map = {}

# Alpha levels from bottom to top
alpha_levels = [0.8, 0.6, 0.4, 0.1]


fig = plt.figure(figsize=(10, 8))
gs = gridspec.GridSpec(2, 2, width_ratios=[4, 1], height_ratios=[1, 4], hspace=0.05, wspace=0.05)
ax_scatter = fig.add_subplot(gs[1, 0])
ax_hist_x = fig.add_subplot(gs[0, 0], sharex=ax_scatter)
ax_hist_y = fig.add_subplot(gs[1, 1], sharey=ax_scatter)

all_damages = []
all_deaths = []
used_labels = set()

for i, filename in enumerate(sorted(os.listdir(folder_path))):
    if '_robustness_' in filename and filename.endswith('.csv'):
        filepath = os.path.join(folder_path, filename)
        df = pd.read_csv(filepath)

        if all(col in df.columns for col in ['Expected Annual Damage', 'Expected Number of Deaths']):
            match = re.search(r'policy_(\d+)_s(\d+)', filename)
            if match:
                policy_id = match.group(1)
                seed_id = match.group(2)
                label = f"Policy {policy_id} (seed {seed_id})"

                if label not in color_map:
                    color_map[label] = policy_colors[len(color_map) % len(policy_colors)]
                color = color_map[label]

                alpha = alpha_levels[len(color_map)-1]  # assign alpha by order

                ax_scatter.scatter(
                    df['Expected Annual Damage'],
                    df['Expected Number of Deaths'],
                    color=color,
                    label=label if label not in used_labels else None,
                    alpha=alpha,
                    s=12,
                    edgecolors='none'
                )
                used_labels.add(label)

                all_damages.extend(df['Expected Annual Damage'])
                all_deaths.extend(df['Expected Number of Deaths'])

# Histograms
ax_hist_x.hist(all_damages, bins=30, color='gray', alpha=0.5)
ax_hist_y.hist(all_deaths, bins=30, orientation='horizontal', color='gray', alpha=0.5)
ax_hist_x.axis('off')
ax_hist_y.axis('off')

# Labels and legend
ax_scatter.set_xlabel('Expected Annual Damage')
ax_scatter.set_ylabel('Expected Number of Deaths')
ax_scatter.grid(True)
plt.suptitle('Joint Distribution of Damage and Deaths (Top 4 Policies, 100.000 Scenarios)', fontsize=14)

ax_scatter.legend(title='Policy', markerscale=4, fontsize='small')

output_folder = 'plots'
os.makedirs(output_folder, exist_ok=True)
save_path = os.path.join(output_folder, 'top_policies_plot.png')
plt.savefig(save_path, facecolor='white', edgecolor='white', bbox_inches='tight', dpi=300)

plt.show()


In [ ]:
import os
import pandas as pd

def evaluate_robustness_with_regret(df, policy_id_col, scenario_id_col, performance_cols, weights):
    regret_df = df.copy()

    for col in performance_cols:
        regret_df[f'{col} Regret'] = regret_df.groupby(scenario_id_col)[col].transform(lambda x: x - x.min())

    grouped = regret_df.groupby(policy_id_col)
    results = []

    for policy, group in grouped:
        metrics = {'Policy': policy}
        for col in performance_cols:
            values = group[col]
            regrets = group[f'{col} Regret']
            metrics[f'{col} Mean'] = values.mean()
            metrics[f'{col} Std'] = values.std()
            metrics[f'{col} Max Regret'] = regrets.max()
            metrics[f'{col} Mean Regret'] = regrets.mean()
        results.append(metrics)

    robustness_df = pd.DataFrame(results)

    for col in performance_cols:
        for stat in ['Mean', 'Std', 'Max Regret', 'Mean Regret']:
            metric_col = f'{col} {stat}'
            min_val = robustness_df[metric_col].min()
            max_val = robustness_df[metric_col].max()
            denom = max_val - min_val if max_val > min_val else 1e-9
            robustness_df[f'{metric_col} (Norm)'] = (robustness_df[metric_col] - min_val) / denom

    score_cols = []
    for col in performance_cols:
        for stat, weight in zip(['Mean', 'Std', 'Max Regret', 'Mean Regret'], weights[col]):
            norm_col = f'{col} {stat} (Norm)'
            weighted_col = f'{col} {stat} Weighted'
            robustness_df[weighted_col] = robustness_df[norm_col] * weight
            score_cols.append(weighted_col)

    robustness_df['Total Score'] = robustness_df[score_cols].sum(axis=1)

    return robustness_df

def analyze_outcomes_folder(folder_path, save_dir='analysis_outputs'):
    os.makedirs(save_dir, exist_ok=True)
    all_dfs = []

    for filename in os.listdir(folder_path):
        if '_outcomes_' in filename and filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)

            policy_id = filename.split('_')[1] + '_' + filename.split('_')[2]
            df['policy'] = policy_id
            df['scenario'] = df.index

            if 'Dike Investment Costs' in df.columns and 'RfR Investment Costs' in df.columns:
                df['Total Investment Costs'] = df['Dike Investment Costs'] + df['RfR Investment Costs']

            if 'Evacuation Costs' in df.columns and 'Expected Annual Damage' in df.columns:
                df['Expected Annual Costs'] = df['Evacuation Costs'] + df['Expected Annual Damage']

            all_dfs.append(df)

    if not all_dfs:
        raise ValueError("No outcome CSV files found with '_outcomes_' in filename.")

    combined_df = pd.concat(all_dfs, ignore_index=True)

    performance_columns = [
        'Expected Number of Deaths',
        'Expected Annual Damage',
        'Evacuation Costs',
        'Total Investment Costs'
    ]

    weights = {
        'Expected Number of Deaths': [1.0, 0.0, 1.0, 0.0],
        'Expected Annual Damage':   [0.75, 0.75, 0.0, 0.5],
        'Evacuation Costs':         [0.5, 0.5, 0.0, 0.25],
        'Total Investment Costs':   [0.25, 0.25, 0.0, 0.0],
    }

    robustness_df = evaluate_robustness_with_regret(
        combined_df,
        policy_id_col='policy',
        scenario_id_col='scenario',
        performance_cols=performance_columns,
        weights=weights
    )

    sorted_df = robustness_df.sort_values('Total Score').reset_index(drop=True)



    return sorted_df, combined_df

def print_robustness_summary(sorted_df, performance_columns):
    for i, row in sorted_df.iterrows():
        print(f"\nRank {i+1} Policy: {row['Policy']}")
        print(f"{'Metric':30} {'Mean':>15} {'Std':>15} {'Max Regret':>15} {'Mean Regret':>15}")
        print("-" * 90)
        for col in performance_columns:
            print(f"{col:30} "
                  f"{row[f'{col} Mean']:15.6g} "
                  f"{row[f'{col} Std']:15.6g} "
                  f"{row[f'{col} Max Regret']:15.6g} "
                  f"{row[f'{col} Mean Regret']:15.6g}")

# Usage
folder_path = 'robustness_results'
sorted_robustness_df, combined_outcomes_df = analyze_outcomes_folder(folder_path)

performance_cols = [
    'Expected Number of Deaths',
    'Expected Annual Damage',
    'Evacuation Costs',
    'Total Investment Costs'
]

print_robustness_summary(sorted_robustness_df, performance_cols)


In [ ]:

combined_outcomes_df.to_csv(os.path.join(folder_path, 'combined_outcomes.csv'), index=False)
sorted_robustness_df.to_csv(os.path.join(folder_path, 'robustness_summary.csv'), index=False)